In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp utility

In [ ]:
#all_norun

In [ ]:
from genetic_musical_generator.random_genome_to_midi import random_genome
from genetic_musical_generator.breed import mutate_crossover, mutate, crossover

from dataclasses import dataclass
from typing import Optional
from random import choices
from time import sleep

In [ ]:
#export
from genetic_musical_generator.random_genome_to_midi import genome2midi
import mido

outport = mido.open_output('mido_out', virtual=True)

In [ ]:
import pickle
from pathlib import Path

In [ ]:
savedir = Path('populations')
savedir.mkdir(exist_ok=True)

In [ ]:
#export
def hear(genome):
    print(genome)
    song = genome2midi(genome)
    for m in song.play():
        outport.send(m)

In [ ]:
e = random_genome(40)
e

In [ ]:
hear(e)

In [ ]:
@dataclass
class Individual:
    genome: str
    rating: Optional[float] = None

In [ ]:
with (savedir/'leo_02.pop').open('rb') as f:
    population = pickle.load(f)
population

In [ ]:
population = []

In [ ]:
pop_size = 10

In [ ]:
expand = False

if len(population)<pop_size:
    population.append(Individual(random_genome(40)))
else:
    try:
        a = b = choices([p.genome for p in population], weights=[p.rating for p in population])[0]
        while a == b:
            b = choices([p.genome for p in population], weights=[p.rating for p in population])[0]
        # population.append(Individual(mutate_crossover(a,b)))
        population.append(Individual(mutate(crossover(a,b), skip_prob=0.01, change_prob=0.02, add_prob=0.01)))
    except TypeError:
        # unrated example present, have to rate first
        pass

for p in population:
    while p.rating is None:
        try:
            hear(p.genome)
            ui = input('Please rate (zero or less to kill, prefix k to keep): ')
            if len(ui) > 0 and ui[0] == 'k':
                expand = True
                ui = ui[1:]
            p.rating = float(ui)
            if expand:
                pop_size += 1
                expand = False
        except ValueError:
            pass
        print()

# remove unfit individuals
population = [p for p in population if p.rating>0]

# remove least fit individuals down to population size
if len(population)>pop_size:
    population = sorted(population, key=lambda p: -p.rating)[:pop_size]

In [ ]:
population

In [ ]:
from genetic_musical_generator.beat_fit import BeatFit, play

In [ ]:
beatfit = BeatFit(genome2midi(population[9].genome))
beatfit.fit(iterations=100)

In [ ]:
play(beatfit.complete)

In [ ]:
beatfit.tempos

In [ ]:
beatfit.loss_map

In [ ]:
for p in sorted(population, key=lambda p: p.rating)[8:]:
    beatfit = BeatFit(genome2midi(p.genome))
    beatfit.fit(iterations=100)
    play(beatfit.complete)

In [ ]:
for p in sorted(population, key=lambda p: p.rating):
    hear(p.genome)
    sleep(4)

In [ ]:
with (savedir/'temp.pop').open('wb') as f:
    pickle.dump(population, f)

with (savedir/'temp.pop').open('rb') as f:
    assert population == pickle.load(f)

- Züchte: choosing 
- Anottating : keep notes on the individuals
- How do breeders really do it?
- Precious individuals

is there a diff between musical trained results?

a few stations

difference different people/sessions...

Style preference or musical experience better predictor of "quality"?

---

"Music student" idea - one lesson a week, practice  :DD

Mid term concert  :D